In [1]:
from pathlib import Path

# Define the expected dataset directory structure
dataset_structure = {
    "Tea leaf dataset": [
        "Healthy",
        "Tea leaf blight",
        "Tea red leaf spot",
        "Tea red scab"
    ]
}

# Simulate verification of folder structure (no actual dataset provided)
dataset_path = Path("D:\Browns\Tea Leaves Project\Tea leaf dataset")
structure_check = {str(dataset_path / category): "Exists" for category in dataset_structure["Tea leaf dataset"]}
structure_check


{'D:\\Browns\\Tea Leaves Project\\Tea leaf dataset\\Healthy': 'Exists',
 'D:\\Browns\\Tea Leaves Project\\Tea leaf dataset\\Tea leaf blight': 'Exists',
 'D:\\Browns\\Tea Leaves Project\\Tea leaf dataset\\Tea red leaf spot': 'Exists',
 'D:\\Browns\\Tea Leaves Project\\Tea leaf dataset\\Tea red scab': 'Exists'}

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

# Set parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = "D:\Browns\Tea Leaves Project\Tea leaf dataset"

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train and validation generators
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


Found 3169 images belonging to 4 classes.
Found 791 images belonging to 4 classes.


In [3]:
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model

def build_model(base_model, name):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions, name=name)
    return model

input_tensor = Input(shape=(*IMG_SIZE, 3))

# Models
efficientnet_base = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)
mobilenet_base = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor)
inception_base = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_tensor)

models = {
    "EfficientNetB0": build_model(efficientnet_base, "EfficientNetB0"),
    "MobileNetV2": build_model(mobilenet_base, "MobileNetV2"),
    "InceptionV3": build_model(inception_base, "InceptionV3")
}


C:\Users\SachithN\AppData\Local\Temp\ipykernel_2012\1614093806.py:17: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet_base = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 26s 3us/step


In [4]:
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def train_model(model, name):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    early_stop = EarlyStopping(patience=5, restore_best_weights=True)
    history = model.fit(train_generator, validation_data=val_generator, epochs=20, callbacks=[early_stop])
    return history, model

histories = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    history, trained_model = train_model(model, name)
    histories[name] = (history, trained_model)



Training EfficientNetB0...


d:\Browns\Tea Leaves Project\.conda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


d:\Browns\Tea Leaves Project\.conda\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


100/100 ━━━━━━━━━━━━━━━━━━━━ 644s 6s/step - accuracy: 0.9219 - loss: 0.2522 - val_accuracy: 0.5740 - val_loss: 1.7941
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 430s 4s/step - accuracy: 0.9734 - loss: 0.0666 - val_accuracy: 0.1568 - val_loss: 2.7393
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 599s 6s/step - accuracy: 0.9886 - loss: 0.0446 - val_accuracy: 0.1732 - val_loss: 1.3768
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 496s 5s/step - accuracy: 0.9932 - loss: 0.0231 - val_accuracy: 0.5740 - val_loss: 3.6997
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 510s 5s/step - accuracy: 0.9755 - loss: 0.0668 - val_accuracy: 0.5740 - val_loss: 9.0544
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 522s 5s/step - accuracy: 0.9858 - loss: 0.0375 - val_accuracy: 0.5487 - val_loss: 1.3105
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 611s 6s/step - accuracy: 0.9952 - loss: 0.0134 - val_accuracy: 0.5740 - val_loss: 2.6245
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 478s 5s/step - accuracy: 0.9920 - loss: 0.0234 - val_accuracy: 0.575

In [5]:
for name, (history, _) in histories.items():
    acc = history.history['val_accuracy'][-1]
    print(f"{name} Final Validation Accuracy: {acc:.4f}")


EfficientNetB0 Final Validation Accuracy: 0.5740
MobileNetV2 Final Validation Accuracy: 0.3552
InceptionV3 Final Validation Accuracy: 0.7914


inseptionv3 is good for this dataset